In [3]:
#importing libraries
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm_notebook as tqdm
import re

In [4]:
#Using proxy chrome driver
driver = webdriver.Chrome('C:/Users/DC/Desktop/chromedriver.exe')

In [5]:
def scrape(address):
    try:
        driver.get(address)
        content=driver.page_source
        soup=BeautifulSoup(content,'html.parser')

        name = [i.text for i in soup.findAll('span', {'class':['a-size-medium a-color-base', 'a-text-normal']})][:-1]
        price = [i.text for i in soup.findAll('span', {'class':['a-price-whole']})]

        image = []
        img_urls = soup.findAll('div', {'class':['a-section', 'aok-relative', 's-image-fixed-height']})
        for url in img_urls:
            try:
                image.append(url.find('img')['src'])
            except:
                pass

        image = list(set(image))
        star = [i.text for i in soup.find_all('i', attrs={'class': re.compile('^a-icon a-icon-star-small.*')})]
        th = min(len(name), len(price), len(image), len(star))
        
        df = pd.DataFrame()
        df['name'] = name[:th]
        df['price'] = price[:th]
        df['url'] = image[:th]
        df['star'] = star[:th]
        
        return df
        
    except Exception as e:
        print(e)
        pass

In [6]:
final_df=pd.DataFrame()

for i in tqdm(range(61)):
    df=scrape(f'https://www.amazon.in/s?k=mobile&i=electronics&rh=n%3A976419031%2Cp_36%3A1318506031&dc&page={i}&qid=1603367829&rnid=1318502031&ref=sr_pg_{i}')
    final_df = pd.concat([final_df, df]).reset_index(drop=True)

C:\Users\DC\anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
final_df

,name,price,url,star
0,"Redmi Note 9 Pro Max (Aurora Blue, 6GB RAM, 64...","15,999",https://m.media-amazon.com/images/I/71wPwmxo2N...,4.1 out of 5 stars
1,"OPPO A15 (Dynamic Black, 3GB RAM, 32GB Storage)","10,990",https://m.media-amazon.com/images/I/61L1ItFgFH...,4.2 out of 5 stars
2,"Samsung Galaxy M21 (Midnight Blue, 4GB RAM, 64...","12,499",https://images-na.ssl-images-amazon.com/images...,4.2 out of 5 stars
3,"Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128GB...","16,499",https://m.media-amazon.com/images/I/91Lr-OxVKj...,4.3 out of 5 stars
4,"Samsung Galaxy M31s (Mirage Blue, 6GB RAM, 128...","18,499",https://m.media-amazon.com/images/I/71JDCbasga...,4.1 out of 5 stars
...,...,...,...,...
980,Honeytecs Professional Photography Light Stand...,"14,997",https://m.media-amazon.com/images/I/51qKB-GMG-...,3.6 out of 5 stars
981,(Renewed) Sennheiser HD 4.50 BT NC Bluetooth W...,"10,800",https://images-eu.ssl-images-amazon.com/images...,4.4 out of 5 stars
982,Sennheiser CX 400BT True Wireless,"18,490","data:image/svg+xml;base64,PHN2ZyB4bWxucz0naHR0...",4.1 out of 5 stars
983,Audio-Technica ATH-E50 Professional in-Ear Mon...,"10,500",https://m.media-amazon.com/images/I/51O0OFy6iz...,2.6 out of 5 stars


In [17]:
final_df.to_csv('Amazon-mobile.csv',index=False,encoding='utf-8')